# 4.3 分组运算

**推荐学习资料：**[Joyful Pandas 第四章](http://joyfulpandas.datawhale.club/Content/ch4.html)

In [1]:
import numpy as np
import pandas as pd

## 1. 分组的一般模式
分组操作在日常生活中使用极其广泛，例如：

* 依据$\color{#FF0000}{性别}$分组，统计全国人口$\color{#00FF00}{寿命}$的$\color{#0000FF}{平均值}$
* 依据$\color{#FF0000}{季节}$分组，对每一个季节的$\color{#00FF00}{温度}$进行$\color{#0000FF}{组内标准化}$
* 依据$\color{#FF0000}{班级}$筛选出组内$\color{#00FF00}{数学分数}$的$\color{#0000FF}{平均值超过80分的班级}$

从上述的几个例子中不难看出，想要实现分组操作，必须明确三个要素：$\color{#FF0000}{分组依据}$、$\color{#00FF00}{数据来源}$、$\color{#0000FF}{操作及其返回结果}$。同时从充分性的角度来说，如果明确了这三方面，就能确定一个分组操作，从而分组代码的一般模式即：
```
df.groupby(分组依据)[数据来源].使用操作
```
例如第一个例子中的代码就应该如下：
```
df.groupby('Gender')['Longevity'].mean()
```

现在返回到学生体测的数据集上，如果想要按照性别统计身高中位数，就可以如下写出：

In [2]:
df = pd.read_csv('learn_pandas.csv')
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

## 2. 分组依据的本质
前面提到的若干例子都是以单一维度进行分组的，比如根据性别，如果现在需要根据多个维度进行分组，该如何做？事实上，只需在`groupby`中传入相应列名构成的列表即可。例如，现希望根据学校和性别进行分组，统计身高的均值就可以如下写出：

In [3]:
df.groupby(['School', 'Gender'])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

目前为止，`groupby`的分组依据都是直接可以从列中按照名字获取的，那如果希望通过一定的复杂逻辑来分组，例如根据学生体重是否超过总体均值来分组，同样还是计算身高的均值。

首先应该先写出分组条件：

In [4]:
condition = df.Weight > df.Weight.mean()

然后将其传入`groupby`中：

In [5]:
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

从索引可以看出，其实最后产生的结果就是按照条件列表中元素的值（此处是`True`和`False`）来分组，下面用随机传入字母序列来验证这一想法：

In [6]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()

a    163.441818
b    164.550000
c    162.011111
Name: Height, dtype: float64

此处的索引就是原先item中的元素，如果传入多个序列进入`groupby`，那么最后分组的依据就是这两个序列对应行的唯一组合：

In [7]:
df.groupby([condition, item])['Height'].mean()

Weight   
False   a    159.877778
        b    158.694737
        c    158.705660
True    a    170.194737
        b    176.911111
        c    171.231579
Name: Height, dtype: float64

由此可以看出，之前传入列名只是一种简便的记号，事实上等价于传入的是一个或多个列，最后分组的依据来自于数据来源组合的unique值，通过`drop_duplicates`就能知道具体的组类别：

In [8]:
df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,Shanghai Jiao Tong University,Female
1,Peking University,Male
2,Shanghai Jiao Tong University,Male
3,Fudan University,Female
4,Fudan University,Male
5,Tsinghua University,Female
9,Peking University,Female
16,Tsinghua University,Male


In [9]:
df.groupby([df['School'], df['Gender']])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

## 3. Groupby对象
能够注意到，最终具体做分组操作时，所调用的方法都来自于`pandas`中的`groupby`对象，这个对象上定义了许多方法，也具有一些方便的属性。

In [10]:
gb = df.groupby(['School', 'Grade'])
gb

通过`ngroups`属性，可以得到分组个数：

In [11]:
gb.ngroups

16

通过`groups`属性，可以返回从$\color{#FF0000}{组名}$映射到$\color{#FF0000}{组索引列表}$的字典：

In [12]:
res = gb.groups
res.keys() # 字典的值由于是索引，元素个数过多，此处只展示字典的键

dict_keys([('Fudan University', 'Freshman'), ('Fudan University', 'Junior'), ('Fudan University', 'Senior'), ('Fudan University', 'Sophomore'), ('Peking University', 'Freshman'), ('Peking University', 'Junior'), ('Peking University', 'Senior'), ('Peking University', 'Sophomore'), ('Shanghai Jiao Tong University', 'Freshman'), ('Shanghai Jiao Tong University', 'Junior'), ('Shanghai Jiao Tong University', 'Senior'), ('Shanghai Jiao Tong University', 'Sophomore'), ('Tsinghua University', 'Freshman'), ('Tsinghua University', 'Junior'), ('Tsinghua University', 'Senior'), ('Tsinghua University', 'Sophomore')])

当`size`作为`DataFrame`的属性时，返回的是表长乘以表宽的大小，但在`groupby`对象上表示统计每个组的元素个数：

In [13]:
gb.size()

School                         Grade    
Fudan University               Freshman      9
                               Junior       12
                               Senior       11
                               Sophomore     8
Peking University              Freshman     13
                               Junior        8
                               Senior        8
                               Sophomore     5
Shanghai Jiao Tong University  Freshman     13
                               Junior       17
                               Senior       22
                               Sophomore     5
Tsinghua University            Freshman     17
                               Junior       22
                               Senior       14
                               Sophomore    16
dtype: int64

通过`get_group`方法可以直接获取所在组对应的行，此时必须知道组的具体名字：

In [14]:
gb.get_group(('Fudan University', 'Freshman'))

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
15,Fudan University,Freshman,Changqiang Yang,Female,156.0,49.0,N,3,2020/1/1,0:05:25
28,Fudan University,Freshman,Gaoqiang Qin,Female,170.2,63.0,N,2,2020/1/7,0:05:24
63,Fudan University,Freshman,Gaofeng Zhao,Female,152.2,43.0,N,2,2019/10/31,0:04:00
70,Fudan University,Freshman,Yanquan Wang,Female,163.5,55.0,N,1,2019/11/19,0:04:07
73,Fudan University,Freshman,Feng Wang,Male,176.3,74.0,N,1,2019/9/26,0:03:31
105,Fudan University,Freshman,Qiang Shi,Female,164.5,52.0,N,1,2019/12/11,0:04:23
108,Fudan University,Freshman,Yanqiang Xu,Female,152.4,38.0,N,1,2019/12/8,0:05:03
157,Fudan University,Freshman,Xiaoli Lv,Female,152.5,45.0,N,2,2019/9/11,0:04:17
186,Fudan University,Freshman,Yanjuan Zhao,Female,NaN,53.0,N,2,2019/10/9,0:04:21


## 4. 常见分组计算函数
常见的分组计算函数如下：`max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod`。

In [15]:
gb = df.groupby('Gender')['Height']
gb.idxmin()

Gender
Female    143
Male      199
Name: Height, dtype: int64

In [16]:
gb.quantile(0.95)

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

这些聚合函数当传入的数据来源包含多个列时，将按照列进行迭代计算：

In [17]:
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0
